# 2 - Model Selection

## Import Train-Test Data

In [1]:
import numpy as np
import pandas as pd

In [2]:
dirname = '../data/processed/'
X_train = pd.read_csv(dirname + 'X_train_trimmed.csv', sep=',')
X_test = pd.read_csv(dirname + 'X_test_trimmed.csv', sep=',')
y_train = pd.read_csv(dirname + 'y_train_trimmed.csv', sep=',')
y_test = pd.read_csv(dirname + 'y_test_trimmed.csv', sep=',')

In [3]:
X_train.shape

(4232, 50)

In [4]:
X_train.columns

Index(['price_reduced_amount', 'year_built', 'lot_sqft', 'sqft', 'baths',
       'garage', 'stories', 'beds', 'central_air', 'central_heat', 'fireplace',
       'rental_property', 'energy_efficient', 'community_security_features',
       'carport', 'dishwasher', 'washer_dryer', 'laundry_room', 'floor_plan',
       'ensuite', 'shopping', 'hardwood_floors', 'high_ceiling',
       'open_floor_plan', 'fenced_yard', 'new_roof', 'front_porch',
       'groundscare', 'basement', 'corner_lot', 'farm', 'ranch', 'forced_air',
       'dining_room', 'family_room', 'view', 'near_outdoors',
       'near_rec_facilities', 'fancy_kitchen', 'type_condo', 'type_land',
       'type_mobile', 'type_multi_family', 'type_other', 'type_single_family',
       'type_townhome', 'season_autumn', 'season_summer', 'season_winter',
       'median_by_pc'],
      dtype='object')

## Instructions (Delete Later)

This notebook should include preliminary and baseline modeling.
- Try as many different models as possible.
- Don't worry about hyperparameter tuning or cross validation here.
- Ideas include:
    - linear regression
    - support vector machines
    - random forest
    - xgboost

In [5]:
# import models and fit

Consider what metrics you want to use to evaluate success.
- If you think about mean squared error, can we actually relate to the amount of error?
- Try root mean squared error so that error is closer to the original units (dollars)
- What does RMSE do to outliers?
- Is mean absolute error a good metric for this problem?
- What about R^2? Adjusted R^2?
- Briefly describe your reasons for picking the metrics you use

In [6]:
# gather evaluation metrics and compare results

**STRETCH**

Even with all the preprocessing we did in Notebook 1, you probably still have a lot of features. Are they all important for prediction?

Investigate some feature selection algorithms (Lasso, RFE, Forward/Backward Selection)
- Perform feature selection to get a reduced subset of your original features
- Refit your models with this reduced dimensionality - how does performance change on your chosen metrics?
- Based on this, should you include feature selection in your final pipeline? Explain

Remember, feature selection often doesn't directly improve performance, but if performance remains the same, a simpler model is often preferrable. 



In [7]:
# perform feature selection 
# refit models
# gather evaluation metrics and compare to the previous step (full feature set)

## Machine Learning

### Linear Regression

In [8]:
from sklearn.preprocessing import MinMaxScaler, RobustScaler, PowerTransformer,\
    PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LinearRegression, Ridge, Lasso

from utils import run_regression

Apply a scaler to the feature data. Actually, some features should be scaled whereas others are Boolean and don't need to be scaled. We tried a few different scalers. `MinMaxScaler()` and `RobustScaler()` yielded similar results whereas the result from `PowerTransformer()` were a little worse than the other two. 

In [9]:
# Get a list of columns to be scaled.
columns = X_train.columns.to_list()
central_air_idx = columns.index('central_air')
features_to_scale = columns[:central_air_idx]
features_to_scale.append('median_by_pc')
print(features_to_scale)

['price_reduced_amount', 'year_built', 'lot_sqft', 'sqft', 'baths', 'garage', 'stories', 'beds', 'median_by_pc']


In [10]:
X_train_fts = X_train[features_to_scale]
X_train_other = X_train.drop(columns=features_to_scale)

X_test_fts = X_test[features_to_scale]
X_test_other = X_test.drop(columns=features_to_scale)

In [11]:
# Choose a scaler.
scaler = MinMaxScaler()
# scaler = RobustScaler()
# scaler = PowerTransformer(method='yeo-johnson')

In [12]:
# Scale the columns that need scaling. Then, recombine with the other columns. 
X_train_sc = np.hstack([
    scaler.fit_transform(X_train_fts), 
    X_train_other.to_numpy()
])

X_test_sc = np.hstack([
    scaler.transform(X_test_fts), 
    X_test_other.to_numpy()
])

Select the k best features. To begin with, there are 50 features. However, many have low correlations with the target and can safely be dropped, leading to simpler models. Some trial and error indicates that k=8 is a good choice.

In [13]:
skb = SelectKBest(f_classif, k=8)

X_train_sc_skb = skb.fit_transform(
    X_train_sc, 
    np.ravel(y_train.to_numpy())
)

X_test_sc_skb = skb.transform(X_test_sc)

Investigate polynomial features. Some trial and error seems to indicate that there is no reason to include polynomial features here; hence, we've set `degree=1` in the following:

In [14]:
poly = PolynomialFeatures(degree=1)

X_train_sc_skb_poly = poly.fit_transform(X_train_sc_skb)
X_test_sc_skb_poly = poly.transform(X_test_sc_skb)

#### Ordinary Least Squares

In [15]:
model = run_regression(
    [X_train_sc_skb_poly, y_train],
    [X_test_sc_skb_poly, y_test],
    LinearRegression()
)

RMSE train: 89252.87062147324
RMSE test: 86063.44290192694
MAE train: 61218.44565217391
MAE test: 61051.7434443657
R**2 train: 0.7656234217026554
R**2 test: 0.7843332803438049
Adj R**2 train: 0.7651238032268912
Adj R**2 test: 0.7829478410312383


#### Ridge

We loop over a few values of the hyperparameter alpha. Doing so doesn't appear to have much effect on the analysis.

In [16]:
alphas = [0.01, 0.1, 1, 10]
for alpha in alphas:
    print(f'alpha = {alpha}')
    model = run_regression(
        [X_train_sc_skb_poly, y_train],
        [X_test_sc_skb_poly, y_test],
        Ridge(alpha=alpha)
    )
    print()

alpha = 0.01
RMSE train: 89251.61714016362
RMSE test: 86063.53324387537
MAE train: 61213.81236779648
MAE test: 61047.03226436391
R**2 train: 0.7656300048992578
R**2 test: 0.7843328275670803
Adj R**2 train: 0.7651304004568356
Adj R**2 test: 0.7829473853458838

alpha = 0.1
RMSE train: 89251.66587880427
RMSE test: 86068.47945877376
MAE train: 61216.95374131726
MAE test: 61053.53005693284
R**2 train: 0.7656297489290367
R**2 test: 0.7843080373500226
Adj R**2 train: 0.765130143940965
Adj R**2 test: 0.7829224358768965

alpha = 1
RMSE train: 89256.47290695073
RMSE test: 86121.85685060041
MAE train: 61248.60254135914
MAE test: 61119.70993924571
R**2 train: 0.7656045022353851
R**2 test: 0.784040421504394
Adj R**2 train: 0.7651048434291602
Adj R**2 test: 0.78265310087166

alpha = 10
RMSE train: 89677.8949676224
RMSE test: 86991.59909339724
MAE train: 61978.95173337249
MAE test: 62135.71752462698
R**2 train: 0.7633858928127034
R**2 test: 0.7796564563685622
Adj R**2 train: 0.7628815046164255
Adj R*

#### Lasso

Again, we loop through several values of alpha to asses the parameter's impact on the analysis. As with the Ridge regression analysis above, changing alpha does not have much effect on the results.

In [17]:
alphas = [0.1, 1, 10]
for alpha in alphas:
    print(f'alpha = {alpha}')
    model = run_regression(
        [X_train_sc_skb_poly, y_train],
        [X_test_sc_skb_poly, y_test],
        Lasso(alpha=alpha)
    )
    print()

alpha = 0.1
RMSE train: 89251.61665544404
RMSE test: 86063.04306307028
MAE train: 61213.377124843384
MAE test: 61046.217569771805
R**2 train: 0.7656300074449535
R**2 test: 0.7843352842546732
Adj R**2 train: 0.7651304030079579
Adj R**2 test: 0.7829498578151958

alpha = 1
RMSE train: 89251.61748062872
RMSE test: 86063.54065307337
MAE train: 61212.599184713836
MAE test: 61045.38765912565
R**2 train: 0.7656300031111714
R**2 test: 0.784332790433563
Adj R**2 train: 0.7651303986649375
Adj R**2 test: 0.7829473479738214

alpha = 10
RMSE train: 89251.69975241416
RMSE test: 86068.81592408966
MAE train: 61204.25330173423
MAE test: 61036.765567275266
R**2 train: 0.7656295710283132
R**2 test: 0.7843063509485279
Adj R**2 train: 0.7651299656610121
Adj R**2 test: 0.7829207386419873



### Support Vector Machine

In [18]:
from sklearn import svm

**For our SVM investigations, we continue to use the dataset from the Linear Regression section above. Recall that this dataset has been scaled. Also, from this dataset, we work with the k=8 best features.**

In the category of SVMs, we first look at a linear kernel. Through trial and error, we find a good value of the hyperparameter C of ~350.

In [19]:
C_values = list(np.arange(300, 500, 50))
for C in C_values:
    print(f'C = {C}')
    model = run_regression(
        [X_train_sc_skb_poly, np.ravel(y_train)],
        [X_test_sc_skb_poly, np.ravel(y_test)],
        svm.SVR(kernel='linear', C=C)
    )
    print()

C = 300
RMSE train: 151912.97717673305
RMSE test: 153606.37420887017
MAE train: 117324.91520322875
MAE test: 118827.16812995951
R**2 train: 0.3210159482213766
R**2 test: 0.3129882304605941
Adj R**2 train: 0.3195685639328858
Adj R**2 test: 0.30857487862201116

C = 350
RMSE train: 147228.81850514354
RMSE test: 148902.34876912585
MAE train: 113270.66337602049
MAE test: 114808.95700469491
R**2 train: 0.3622426429668929
R**2 test: 0.35442188734133084
Adj R**2 train: 0.36088314125839027
Adj R**2 test: 0.3502747046047655

C = 400
RMSE train: 142505.90961252525
RMSE test: 144114.63437577695
MAE train: 109445.80963353819
MAE test: 111101.88559941734
R**2 train: 0.4025032153445478
R**2 test: 0.39526950467266375
Adj R**2 train: 0.40122953674153994
Adj R**2 test: 0.3913847263300898

C = 450
RMSE train: 138311.66592826872
RMSE test: 139810.5490729341
MAE train: 105942.3587122938
MAE test: 107600.71059068604
R**2 train: 0.437156768267597
R**2 test: 0.43085151540516786
Adj R**2 train: 0.4359569603363

Second, we consider the 'rbf' kernel. Through trial and error, we found the good values of the hyperparameters C=100,000 and gamma=2. The hyperparameter epsilon appears to have little effect on the analysis.  

In [20]:
eps_values = [0.01, 0.1, 1, 10]
for eps in eps_values:
    print(f'eps = {eps}')
    model = run_regression(
        [X_train_sc_skb_poly, np.ravel(y_train)],
        [X_test_sc_skb_poly, np.ravel(y_test)],
        svm.SVR(kernel='rbf', C=100_000, gamma=2, epsilon=eps)
    )
    print()

eps = 0.01
RMSE train: 88485.36479919084
RMSE test: 85420.65849326775
MAE train: 53971.87252152956
MAE test: 53702.887503314596
R**2 train: 0.7696370057908679
R**2 test: 0.7875427619957461
Adj R**2 train: 0.7691459430367509
Adj R**2 test: 0.7861779403383312

eps = 0.1
RMSE train: 88485.33318969741
RMSE test: 85420.62691517737
MAE train: 53971.87448307618
MAE test: 53702.8897149648
R**2 train: 0.7696371703752884
R**2 test: 0.7875429190770129
Adj R**2 train: 0.7691461079720145
Adj R**2 test: 0.7861780984286854

eps = 1
RMSE train: 88485.01705532643
RMSE test: 85420.31121135446
MAE train: 53971.90173553646
MAE test: 53702.91418982472
R**2 train: 0.7696388164215997
R**2 test: 0.7875444895030593
Adj R**2 train: 0.7691477575271881
Adj R**2 test: 0.7861796789431218

eps = 10
RMSE train: 88482.15046823306
RMSE test: 85416.85500086556
MAE train: 53972.51832676385
MAE test: 53702.75142060538
R**2 train: 0.7696537418795175
R**2 test: 0.7875616815759572
Adj R**2 train: 0.7691627148015724
Adj R**2 

### Random Forest

### XGBoost